# Autoencoder Model on MNIST dataset

In [1]:
import torch;

torch.manual_seed(0)
import torch.nn as nn
import torch.nn.functional as F
import torch.utils
import torch.distributions
import torchvision
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.dpi'] = 200

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Model implementation

### Encoder class

In [3]:
class Encoder(nn.Module):
    def __init__(self, latent_dims):
        super(Encoder, self).__init__()
        self.linear1 = nn.Linear(784, 512)
        self.linear2 = nn.Linear(512, latent_dims)

    def forward(self, x):
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.linear1(x))
        x = self.linear2(x)
        return x

### Decoder class

In [4]:
class Decoder(nn.Module):
    def __init__(self, latent_dims):
        super(Decoder, self).__init__()
        self.linear1 = nn.Linear(latent_dims, 512)
        self.linear2 = nn.Linear(512, 784)

    def forward(self, z):
        z = F.relu(self.linear1(z))
        z = torch.sigmoid(self.linear2(z))
        return z.view((-1, 1, 28, 28))

### Autoencoder class

In [5]:
class Autoencoder(nn.Module):
    def __init__(self, latent_dims):
        super(Autoencoder, self).__init__()
        self.encoder = Encoder(latent_dims)
        self.decoder = Decoder(latent_dims)

    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)

## Training

In [6]:
def train(model, data, epochs=20):
    optim = torch.optim.Adam(model.parameters())
    criterion = nn.MSELoss()
    for epoch in range(epochs):
        for x, y in data:
            x = x.to(device)
            optim.zero_grad()
            x_hat = model(x)
            loss = criterion(x_hat, x)
            loss.backward()
            optim.step()
        print(f'epoch:{epoch} loss:{loss}')
    return model

## Dataset

In [7]:
from torch.utils.data import DataLoader
data = DataLoader(
    torchvision.datasets.MNIST(
        '.pytorch/autoencodermnist', transform=torchvision.transforms.ToTensor(), download=True
    ),
    batch_size=128,
    shuffle=True
)

In [8]:
latent_dims = 2
ae = Autoencoder(latent_dims).to(device)
autoencoder = train(ae, data)

epoch:0 loss:0.048180557787418365
epoch:1 loss:0.043877262622117996
epoch:2 loss:0.04656091332435608
epoch:3 loss:0.040888793766498566
epoch:4 loss:0.04371849447488785
epoch:5 loss:0.04414192587137222
epoch:6 loss:0.04103892669081688
epoch:7 loss:0.04216746613383293
epoch:8 loss:0.04034364968538284
epoch:9 loss:0.044953975826501846
epoch:10 loss:0.04258110374212265
epoch:11 loss:0.0404970645904541
epoch:12 loss:0.042203404009342194
epoch:13 loss:0.043764542788267136
epoch:14 loss:0.042052146047353745
epoch:15 loss:0.04408789798617363
epoch:16 loss:0.04196309670805931
epoch:17 loss:0.041984669864177704
epoch:18 loss:0.03974001854658127
epoch:19 loss:0.04197254776954651


## Visualize latent space